
Text Generation is a type of Language Modelling problem. Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. In this notebook, I will explain how to create a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network. 

### Generating News headlines 

In this kernel, I will be using the dataset of [New York Times Comments and Headlines](https://www.kaggle.com/aashita/nyt-comments) to train a text generation language model which can be used to generate News Headlines


## 1. Import the libraries

As the first step, we need to import the required libraries:

In [ ]:
# !pip install tensorflow

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

# set seeds for reproducability
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(2)
seed(5555)

import pandas as pd
import numpy as np
import string

import warnings
warnings.filterwarnings("ignore")

## 2. Load the dataset

Load the dataset of news headlines

In [4]:
article_df = pd.read_csv('ArticlesMarch2018.csv')
article_df.head(5)

,articleID,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,5a974697410cf7000162e8a4,By BINYAMIN APPELBAUM,article,"Virtual Coins, Real Resources","['Bitcoin (Currency)', 'Electric Light and Pow...",1,Business,1,2018-03-01 00:17:22,Economy,America has a productivity problem. One explan...,The New York Times,News,https://www.nytimes.com/2018/02/28/business/ec...,1207
1,5a974be7410cf7000162e8af,By HELENE COOPER and ERIC SCHMITT,article,U.S. Advances Military Plans for North Korea,"['United States Defense and Military Forces', ...",1,Washington,11,2018-03-01 00:40:01,Asia Pacific,The American military is looking at everything...,The New York Times,News,https://www.nytimes.com/2018/02/28/world/asia/...,1215
2,5a9752a2410cf7000162e8ba,By THE EDITORIAL BOARD,article,Mr. Trump and the ‘Very Bad Judge’,"['Trump, Donald J', 'Curiel, Gonzalo P', 'Unit...",1,Editorial,26,2018-03-01 01:08:46,Unknown,Can you guess which man is the model public se...,The New York Times,Editorial,https://www.nytimes.com/2018/02/28/opinion/tru...,1043
3,5a975310410cf7000162e8bd,By JAVIER C. HERNÁNDEZ,article,"To Erase Dissent, China Bans Pooh Bear and ‘N’","['China', 'Xi Jinping', 'Term Limits (Politica...",1,Foreign,1,2018-03-01 01:10:35,Asia Pacific,Censors swung into action after Mr. Xi’s bid t...,The New York Times,News,https://www.nytimes.com/2018/02/28/world/asia/...,1315
4,5a975406410cf7000162e8c3,"By JESSE DRUCKER, KATE KELLY and BEN PROTESS",article,Loans Flowed to Kushner Cos. After Visits to t...,"['Kushner, Jared', 'Kushner Cos', 'United Stat...",1,Business,1,2018-03-01 01:14:41,Unknown,"Apollo, the private equity firm, and Citigroup...",The New York Times,News,https://www.nytimes.com/2018/02/28/business/ja...,1566


In [9]:
all_headlines = list([h for h in article_df['headline'].values if h != "Unknown"])

len(all_headlines)

1250

## 3. Dataset preparation

### 3.1 Dataset cleaning 

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words. 

In [10]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['virtual coins real resources',
 'us advances military plans for north korea',
 'mr trump and the very bad judge',
 'to erase dissent china bans pooh bear and n',
 'loans flowed to kushner cos after visits to the white house',
 'china envoy intends to ease trade tensions',
 'president trumps contradictory and sometimes false comments about gun policy to lawmakers',
 'classic letter puzzle',
 'silicon valley disruption in an australian school',
 'the assassination of gianni versace episode 6 a nothing man']

### 3.2 Generating Sequence of N-gram Tokens

Language modelling requires a sequence input data, as given a sequence (of words/tokens) the aim is the predict next word/token.  

The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens. 


In [11]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    
    # getting coresponding index of words in the sentence
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        
        # making combination of words
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words


inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[1119, 1120],
 [1119, 1120, 116],
 [1119, 1120, 116, 1121],
 [31, 1122],
 [31, 1122, 589],
 [31, 1122, 589, 392],
 [31, 1122, 589, 392, 7],
 [31, 1122, 589, 392, 7, 61],
 [31, 1122, 589, 392, 7, 61, 70],
 [117, 10]]

In [17]:
total_words

3582

In the above output [1119, 1120], [1119, 1120,116], [1119, 1120, 116, 1121] and so on represents the ngram phrases generated from the input data. where every integer corresponds to the index of a particular word in the complete vocabulary of words present in the text. For example

**Headline:** i stand  with the shedevils  
**Ngrams:** | **Sequence of Tokens**

<table>
<tr><td>Ngram </td><td> Sequence of Tokens</td></tr>
<tr> <td>i stand </td><td> [30, 507] </td></tr>
<tr> <td>i stand with </td><td> [30, 507, 11] </td></tr>
<tr> <td>i stand with the </td><td> [30, 507, 11, 1] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [30, 507, 11, 1, 975] </td></tr>
</table>



### 3.3 Padding the Sequences and obtain Variables : Predictors and Target

Now that we have generated a data-set which contains sequence of tokens, it is possible that different sequences have different lengths. Before starting training the model, we need to pad the sequences and make their lengths equal. We can use pad_sequence function of Kears for this purpose. To input this data into a learning model, we need to create predictors and label. We will create N-grams sequence as predictors and the next word of the N-gram as label. For example:


Headline:  they are learning data science

<table>
<tr><td>PREDICTORS </td> <td>           LABEL </td></tr>
<tr><td>they                   </td> <td>  are</td></tr>
<tr><td>they are               </td> <td>  learning</td></tr>
<tr><td>they are learning      </td> <td>  data</td></tr>
<tr><td>they are learning data </td> <td>  science</td></tr>
</table>

In [12]:
def generate_padded_sequences(input_sequences):
    
    # finding the length of the longest sentence
    max_sequence_len = max([len(x) for x in input_sequences])
    
    # putting zeros to make length of all sentences same
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    
    # selecting the last column as label , and all before that as predictior
    predictors, label = input_sequences[:,:-1] , input_sequences[:,-1]
    
    # making label as categorical
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [15]:
predictors

array([[   0,    0,    0, ...,    0,    0, 1119],
       [   0,    0,    0, ...,    0, 1119, 1120],
       [   0,    0,    0, ..., 1119, 1120,  116],
       ...,
       [   0,    0,    0, ...,  979,  151,  386],
       [   0,    0,    0, ...,    0,    0, 3581],
       [   0,    0,    0, ...,    0, 3581,    5]])

In [16]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [37]:
def create_model(max_sequence_len, total_words):
    
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length = input_len))
    
    # Add Hidden Layer 1
    # model.add(SimpleRNN(200))
    model.add(LSTM(200))
    # model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation = 'softmax'))

    # compileing model
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    
    return model

model = create_model(max_sequence_len, total_words)
# model.summary()

Lets train our model now

In [38]:
len(predictors)

8057

In [42]:
model.fit(predictors, label, epochs = 50, verbose = 1)

Epoch 1/50
252/252 [==============================] - 4s 15ms/step - loss: 1.3867
Epoch 2/50
252/252 [==============================] - 4s 15ms/step - loss: 1.3502
Epoch 3/50
252/252 [==============================] - 4s 15ms/step - loss: 1.3218
Epoch 4/50
252/252 [==============================] - 4s 17ms/step - loss: 1.2861
Epoch 5/50
252/252 [==============================] - 5s 18ms/step - loss: 1.2534
Epoch 6/50
252/252 [==============================] - 5s 18ms/step - loss: 1.2238
Epoch 7/50
252/252 [==============================] - 5s 18ms/step - loss: 1.1918
Epoch 8/50
252/252 [==============================] - 5s 18ms/step - loss: 1.1648
Epoch 9/50
252/252 [==============================] - 5s 18ms/step - loss: 1.1370
Epoch 10/50
252/252 [==============================] - 5s 19ms/step - loss: 1.1079
Epoch 11/50
252/252 [==============================] - 5s 19ms/step - loss: 1.0778
Epoch 12/50
252/252 [==============================] - 5s 19ms/step - loss: 1.0525
Epoch 13/50
2

## 5. Generating the text 

Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.


In [43]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    
    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen = max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose = 0)[0])
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+ output_word
    return seed_text.title()

## 6. Some Results

In [45]:
nx = 6
print (generate_text("united states", nx, model, max_sequence_len))
print (generate_text("president trump",nx, model, max_sequence_len))
print (generate_text("donald trump", nx, model, max_sequence_len))
print (generate_text("india and china", nx, model, max_sequence_len))
print (generate_text("new york", nx, model, max_sequence_len))
print (generate_text("science and technology", nx, model, max_sequence_len))

United States Leaves Trump To His Instinct Of
President Trump Pushes Tariffs Latin America Links Up
Donald Trump Man At War Street As When
India And China On One Week At A Time
New York Forgets Its Juvenile Lifers Over Trump
Science And Technology Trump Latin And Place Of Brokering
